In [17]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import  cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

In [19]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two


In [26]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list):continue
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)== 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x)== 'None' else x.split(' ')[0])
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)== 'None' else int(str(x).split('cm')[0].replace(' ','')))


In [0]:
def run_model (model, feats):
  X = df[feats].values
  y = df.price_value.values

  scores = cross_val_score(model, X, y, cv = 3, scoring='neg_mean_absolute_error')
  return np. mean(scores), np.std(scores)

In [30]:
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','param_wersja__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']

xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}

model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[14:26:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:26:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:26:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

In [32]:
 def obj_func(params):
   print("Training with params: ")
   print(params)

   mean_mae, score_std = run_model(xgb.XGBRegressor(**params),feats)
   return{ 'loss': np.abs(mean_mae), 'status': STATUS_OK}

#space
xgb_reg_params =  {
       'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
       'max depth':        hp.choice('max_depth',    np.arange(5, 16, 1, dtype=int)),
       'subsample':        hp.quniform('subsample',0.5, 1, 0.05),
       'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
       'objective':        'reg:sqarederror',
       'n_estimators':     100,
       'seed':0
}

##run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

Training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.25, 'max depth': 8, 'n_estimators': 100, 'objective': 'reg:sqarederror', 'seed': 0, 'subsample': 0.55}
  0%|          | 0/25 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
xgboost.core.XGBoostError: [14:34:15] /workspace/src/objective/objective.cc:23: Unknown objective function reg:sqarederror
Stack trace:
  [bt] (0) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(xgboost::ObjFunction::Create(std::string const&)+0x6d0) [0x7f85da4a7070]
  [bt] (1) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(xgboost::LearnerImpl::LazyInitModel()+0x3c1) [0x7f85da44cff1]
  [bt] (2) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(XGBoosterUpdateOneIter+0x52) [0x7f85da3b4ac2]
  [bt] (3) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call_unix64+0x4c) [0x7f860fb98dae]
  [bt] (4) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call+0x22f) [0x7f860fb9871f]
  [bt] (5) /usr/lib/python3.6/lib-dynload/_ctypes.cpython-36m-x86_

Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.3, 'max depth': 5, 'n_estimators': 100, 'objective': 'reg:sqarederror', 'seed': 0, 'subsample': 0.6000000000000001}
  4%|▍         | 1/25 [00:00<00:09,  2.65it/s, best loss: nan]

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
xgboost.core.XGBoostError: [14:34:16] /workspace/src/objective/objective.cc:23: Unknown objective function reg:sqarederror
Stack trace:
  [bt] (0) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(xgboost::ObjFunction::Create(std::string const&)+0x6d0) [0x7f85da4a7070]
  [bt] (1) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(xgboost::LearnerImpl::LazyInitModel()+0x3c1) [0x7f85da44cff1]
  [bt] (2) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(XGBoosterUpdateOneIter+0x52) [0x7f85da3b4ac2]
  [bt] (3) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call_unix64+0x4c) [0x7f860fb98dae]
  [bt] (4) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call+0x22f) [0x7f860fb9871f]
  [bt] (5) /usr/lib/python3.6/lib-dynload/_ctypes.cpython-36m-x86_

Training with params: 
{'colsample_bytree': 0.5, 'learning_rate': 0.25, 'max depth': 14, 'n_estimators': 100, 'objective': 'reg:sqarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.1, 'max depth': 8, 'n_estimators': 100, 'objective': 'reg:sqarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max depth': 13, 'n_estimators': 100, 'objective': 'reg:sqarederror', 'seed': 0, 'subsample': 0.6000000000000001}
 16%|█▌        | 4/25 [00:01<00:07,  2.72it/s, best loss: nan]

...

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
xgboost.core.XGBoostError: [14:34:23] /workspace/src/objective/objective.cc:23: Unknown objective function reg:sqarederror
Stack trace:
  [bt] (0) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(xgboost::ObjFunction::Create(std::string const&)+0x6d0) [0x7f85da4a7070]
  [bt] (1) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(xgboost::LearnerImpl::LazyInitModel()+0x3c1) [0x7f85da44cff1]
  [bt] (2) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(XGBoosterUpdateOneIter+0x52) [0x7f85da3b4ac2]
  [bt] (3) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call_unix64+0x4c) [0x7f860fb98dae]
  [bt] (4) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call+0x22f) [0x7f860fb9871f]
  [bt] (5) /usr/lib/python3.6/lib-dynload/_ctypes.cpython-36m-x86_

Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.2, 'max depth': 8, 'n_estimators': 100, 'objective': 'reg:sqarederror', 'seed': 0, 'subsample': 0.65}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.15000000000000002, 'max depth': 10, 'n_estimators': 100, 'objective': 'reg:sqarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.2, 'max depth': 9, 'n_estimators': 100, 'objective': 'reg:sqarederror', 'seed': 0, 'subsample': 0.65}
 92%|█████████▏| 23/25 [00:08<00:00,  2.70it/s, best loss: nan]

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
xgboost.core.XGBoostError: [14:34:24] /workspace/src/objective/objective.cc:23: Unknown objective function reg:sqarederror
Stack trace:
  [bt] (0) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(xgboost::ObjFunction::Create(std::string const&)+0x6d0) [0x7f85da4a7070]
  [bt] (1) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(xgboost::LearnerImpl::LazyInitModel()+0x3c1) [0x7f85da44cff1]
  [bt] (2) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(XGBoosterUpdateOneIter+0x52) [0x7f85da3b4ac2]
  [bt] (3) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call_unix64+0x4c) [0x7f860fb98dae]
  [bt] (4) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call+0x22f) [0x7f860fb9871f]
  [bt] (5) /usr/lib/python3.6/lib-dynload/_ctypes.cpython-36m-x86_

Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.1, 'max depth': 14, 'n_estimators': 100, 'objective': 'reg:sqarederror', 'seed': 0, 'subsample': 0.6000000000000001}
100%|██████████| 25/25 [00:09<00:00,  2.71it/s, best loss: nan]


AssertionError: ignored

In [33]:
def obj_func(params):
  print("Training with params: ")
  print(params)
  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)
  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

# space
xgb_reg_params = {
'learning_rate':    hp.choice('learning_rate', np.arange(0.05,0.31, 0.05)),
'max_depth':        hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
'subsample':        hp.quniform('subsample', 0.5, 1, 0.05),
'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
'objective':        'reg:squarederror',
'n_estimators':     100,
'seed':             0
}

## run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)
best

Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.25, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Traini

{'colsample_bytree': 0.8,
 'learning_rate': 1,
 'max_depth': 10,
 'subsample': 0.9500000000000001}